# Experiment 013: LOO Ensemble (Fixed Fold Structure)

**CRITICAL FIX**: Revert to LOO validation (24 folds for task 0, 13 folds for task 1).

exp_012 FAILED because GroupKFold changed the fold structure to 5 folds.
The evaluation metric expects the ORIGINAL LOO fold structure.

**Key changes:**
1. Use ORIGINAL LOO utility functions (NOT GroupKFold)
2. Keep MLP + GBDT ensemble architecture
3. Template-compliant submission format

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import lightgbm as lgb
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

CUDA available: True
GPU: NVIDIA H100 80GB HBM3


In [2]:
# --- UTILITY FUNCTIONS (ORIGINAL LOO - NOT GROUPKFOLD) ---
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

# Load Spange descriptors
Spange = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv')
print(f"Spange: {Spange.shape}")
Spange_dict = {row['SOLVENT NAME']: row.drop('SOLVENT NAME').values for _, row in Spange.iterrows()}

Spange: (26, 14)


In [3]:
# --- ORIGINAL LOO UTILITY FUNCTIONS (REQUIRED FOR SUBMISSION) ---
# DO NOT USE GROUPKFOLD - it breaks the submission format!
# Task 0 expects 24 folds (one per solvent)
# Task 1 expects 13 folds (one per solvent ramp)

def generate_leave_one_out_splits(X, Y):
    """Leave-One-Solvent-Out for single solvent data (24 folds)."""
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    """Leave-One-Ramp-Out for full data (13 folds)."""
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & 
                 (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Verify fold counts
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")
print(f"Single solvent folds: {len(list(generate_leave_one_out_splits(X_single, Y_single)))}")
print(f"Full data folds: {len(list(generate_leave_one_ramp_out_splits(X_full, Y_full)))}")

Single solvent folds: 24
Full data folds: 13


In [4]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [5]:
# --- MLP ARCHITECTURE (TOP KERNEL - NO SIGMOID) ---
class TopKernelMLP(nn.Module):
    """MLP with BatchNorm + ReLU + Dropout, LINEAR output (no Sigmoid)."""
    def __init__(self, input_dim, hidden_dims=[128, 64, 32], output_dim=3, dropout=0.1):
        super().__init__()
        layers = []
        layers.append(nn.BatchNorm1d(input_dim))
        
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            prev_dim = h_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        # Handle batch_size=1 for BatchNorm
        if x.size(0) == 1 and self.training:
            self.eval()
            out = self.network(x)
            self.train()
            return out
        return self.network(x)

print("TopKernelMLP defined")

TopKernelMLP defined


In [6]:
# --- MLP + GBDT ENSEMBLE MODEL ---
class TopKernelEnsemble(BaseModel):
    """Ensemble of MLP + XGBoost + RandomForest + LightGBM.
    
    Architecture:
    - MLP: [128, 64, 32], NO Sigmoid, 100 epochs, lr=1e-3, dropout=0.1
    - XGBoost: n_estimators=300, max_depth=6
    - RandomForest: n_estimators=300, max_depth=15
    - LightGBM: n_estimators=300
    - Weights: [0.4, 0.2, 0.2, 0.2]
    """
    
    def __init__(self, data='single'):
        super().__init__()
        self.data = data
        self.scaler = StandardScaler()
        self.mlp = None
        self.xgb_models = []
        self.rf_model = None
        self.lgb_models = []
        self.weights = [0.4, 0.2, 0.2, 0.2]
    
    def _get_features(self, X):
        features = []
        if self.data == 'single':
            for _, row in X.iterrows():
                solvent = row['SOLVENT NAME']
                spange = Spange_dict.get(solvent, np.zeros(13))
                feat = np.concatenate([[row['Residence Time'], row['Temperature']], spange])
                features.append(feat)
        else:
            for _, row in X.iterrows():
                solvent_a = row['SOLVENT A NAME']
                solvent_b = row['SOLVENT B NAME']
                pct_b = row['SolventB%'] / 100.0
                spange_a = Spange_dict.get(solvent_a, np.zeros(13))
                spange_b = Spange_dict.get(solvent_b, np.zeros(13))
                spange_mix = (1 - pct_b) * spange_a + pct_b * spange_b
                feat = np.concatenate([[row['Residence Time'], row['Temperature'], pct_b], spange_mix])
                features.append(feat)
        return np.array(features)
    
    def train_model(self, X_train, y_train):
        X_feat = self._get_features(X_train)
        y_np = y_train.values
        X_scaled = self.scaler.fit_transform(X_feat)
        
        # Train MLP
        input_dim = X_scaled.shape[1]
        self.mlp = TopKernelMLP(input_dim, hidden_dims=[128, 64, 32], output_dim=3, dropout=0.1).to(device)
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        y_tensor = torch.tensor(y_np, dtype=torch.double).to(device)
        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        optimizer = torch.optim.Adam(self.mlp.parameters(), lr=1e-3)
        criterion = nn.MSELoss()
        
        self.mlp.train()
        for epoch in range(100):
            for batch_X, batch_y in loader:
                optimizer.zero_grad()
                pred = self.mlp(batch_X)
                loss = criterion(pred, batch_y)
                loss.backward()
                optimizer.step()
        
        # Train XGBoost
        self.xgb_models = []
        for i in range(3):
            model = xgb.XGBRegressor(n_estimators=300, max_depth=6, learning_rate=0.05,
                                     subsample=0.8, colsample_bytree=0.8, random_state=42, verbosity=0)
            model.fit(X_scaled, y_np[:, i])
            self.xgb_models.append(model)
        
        # Train RandomForest
        self.rf_model = MultiOutputRegressor(
            RandomForestRegressor(n_estimators=300, max_depth=15, random_state=42, n_jobs=-1))
        self.rf_model.fit(X_scaled, y_np)
        
        # Train LightGBM
        self.lgb_models = []
        for i in range(3):
            model = lgb.LGBMRegressor(n_estimators=300, learning_rate=0.05, num_leaves=31,
                                      max_depth=-1, random_state=42, verbosity=-1)
            model.fit(X_scaled, y_np[:, i])
            self.lgb_models.append(model)
    
    def predict(self, X_test):
        X_feat = self._get_features(X_test)
        X_scaled = self.scaler.transform(X_feat)
        
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
            mlp_pred = self.mlp(X_tensor).cpu().numpy()
        
        xgb_pred = np.column_stack([m.predict(X_scaled) for m in self.xgb_models])
        rf_pred = self.rf_model.predict(X_scaled)
        lgb_pred = np.column_stack([m.predict(X_scaled) for m in self.lgb_models])
        
        final_pred = (self.weights[0] * mlp_pred + self.weights[1] * xgb_pred +
                      self.weights[2] * rf_pred + self.weights[3] * lgb_pred)
        final_pred = np.clip(final_pred, 0, 1)
        
        return torch.tensor(final_pred)

print("TopKernelEnsemble defined")

TopKernelEnsemble defined


In [7]:
# --- QUICK VALIDATION TEST (BEFORE TEMPLATE CELLS) ---
print("Quick test of TopKernelEnsemble with LOO...")
X_test, Y_test = load_data("single_solvent")

errors = []
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_test, Y_test)):
    if i >= 3: break
    solvent = test_X['SOLVENT NAME'].iloc[0]
    model = TopKernelEnsemble(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    print(f"Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nQuick test MAE (3 folds): {np.mean(errors):.4f}")

Quick test of TopKernelEnsemble with LOO...


Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1856


Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.1221


Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0305

Quick test MAE (3 folds): 0.1127


In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = TopKernelEnsemble(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:04,  4.12s/it]

2it [00:08,  4.16s/it]

3it [00:12,  4.12s/it]

4it [00:16,  4.13s/it]

...

21it [01:28,  4.22s/it]

22it [01:32,  4.23s/it]

23it [01:36,  4.20s/it]

24it [01:41,  4.21s/it]

24it [01:41,  4.21s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = TopKernelEnsemble(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:06,  6.73s/it]

2it [00:13,  6.68s/it]

3it [00:20,  6.73s/it]

4it [00:26,  6.74s/it]

...

10it [01:08,  6.89s/it]

11it [01:15,  6.99s/it]

12it [01:22,  7.10s/it]

13it [01:29,  7.13s/it]

13it [01:29,  6.91s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################